In [18]:
# Scrapes the Latin alphabet Politika.rs website news archive > XML output
# Pokušaj da popravim da mi vrti stranice u mesecima

from bs4 import BeautifulSoup
import requests

agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

politika = "https://www.politika.rs"
arhiva = "https://www.politika.rs/sr/articles/archive/"
end = "Sadržaj ove stranice privremeno nije dostupan zbog podizanja novog portala. Uskoro ce svi sadržaji postati dostupni. Hvala na strpljenju."

# Enter year to parse, post-2005
year = 2016

# Determine link to archive
for month in range (1,13):
    page = 1
    
    if month<10:
        url = arhiva + str(year) +"/0" + str(month) + "/14/page:" + str(page) + "?url="
    else:
        url = arhiva + str(year) + "/" + str(month) + "/14/page:" + str(page) + "?url="

    checkresponse = requests.get(url, headers = agent)
    checksoup = BeautifulSoup(checkresponse.text, 'html.parser')
    check = checksoup.find('h1')
    checkend = check.text.strip()

    print(url)
    
    # Write text to a new file
    outpath = str(year) + "-" + str(month) + "-14 Politika.xml"
    print("Writing text to:", outpath)
    f = open(outpath, "w", encoding='utf-8', newline='')

    # Add TEI header
    print('Adding header')
    with open("TEI header.txt") as header:
        for line in header:
            f.write(line)
    
    # Iterate pages until an error page is encountered
    while checkend != end:
        print('Page', str(page))
        response = requests.get(url, headers = agent)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find('body').find_all('a', href=True)
        count = 0
        
        # Grab text from linked articles
        for x in links:
            if "clanak" in x['href']:
                newurl = politika + x['href']
                newresponse = requests.get(newurl, headers = agent)
                newsoup = BeautifulSoup(newresponse.text, 'html.parser')
                naziv = newsoup.find('h1')
                vest = newsoup.find('div', {"id": "text-holder"})

                # Since they're double linked in the source code
                if (count % 2 == 0):
                    txt1 = naziv.text.strip() + "\n"
                    txt2 = vest.text.strip() + "\n\n"
                    txt3 = txt1 + txt2
                    f.write(txt3)
                count += 1
            
        page += 1
        if month<10:
            url = arhiva + str(year) +"/0" + str(month) + "/14/page:" + str(page) + "?url="
        else:
            url = arhiva + str(year) + "/" + str(month) + "/14/page:" + str(page) + "?url="

        checkresponse = requests.get(url, headers = agent)
        checksoup = BeautifulSoup(checkresponse.text, 'html.parser')
        check = checksoup.find('h1')
        checkend = check.text.strip()

    # Add TEI closed tags at the end
    print('Adding footer', flush=True)
    with open("TEI closed tags.txt") as footer:
        for line in footer:
            f.write(line)        

    print(url)

https://www.politika.rs/sr/articles/archive/2016/01/14/page:1?url=
Writing text to: 2016-1-14 Politika.xml
Adding header
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Adding footer
https://www.politika.rs/sr/articles/archive/2016/01/14/page:9?url=
https://www.politika.rs/sr/articles/archive/2016/02/14/page:1?url=
Writing text to: 2016-2-14 Politika.xml
Adding header
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Adding footer
https://www.politika.rs/sr/articles/archive/2016/02/14/page:8?url=
https://www.politika.rs/sr/articles/archive/2016/03/14/page:1?url=
Writing text to: 2016-3-14 Politika.xml
Adding header
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Adding footer
https://www.politika.rs/sr/articles/archive/2016/03/14/page:8?url=
https://www.politika.rs/sr/articles/archive/2016/04/14/page:1?url=
Writing text to: 2016-4-14 Politika.xml
Adding header
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Adding footer
https://www.politika.rs/sr/articles/archive/2016/04/14/p

In [3]:
#Scuffed word count 

import os, glob

total = 0
filecount = 0

for filename in glob.glob('200*.xml'):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        filecount = filecount + 1
        data = f.read()
        words = data.split()
        total = total + len(words)
        
total = total - filecount*55       # 55 is the WC in the two TEI files
print(total)

1730901
